In [1]:
date_wanted = '2017-06-06' #YYYY-MM-DD or all
job_page = 'Aesop.htm'

In [2]:
from bs4 import BeautifulSoup
import re
import json
import pandas as pd
import urllib

In [3]:
nearby_high_schools = open('nearby_high_schools.txt').read().split('\n')
nearby_k_8 = open('nearby_k_8.txt').read().split('\n')
train_bus_high_schools = open('train_bus_high_schools.txt').read().split('\n')
train_bus_k_8 = open('train_bus_k_8.txt').read().split('\n')

In [4]:
all_nearby = nearby_high_schools + nearby_k_8
all_train_bus = train_bus_high_schools + train_bus_k_8
all_schools = nearby_high_schools + nearby_k_8 + train_bus_high_schools + train_bus_k_8

In [5]:
zips = pd.read_csv('Neighboorhood_zips.csv',header=None,names=['Zip','Neighborhoods'])
staff_websites = pd.read_csv('staff_websites.csv')

In [6]:
def find_subject(school, name):
    url = staff_websites[staff_websites['School'] == school]['Staff Website'].values
    if len(url) != 1:
        return ''
    script = urllib.urlopen(url[0])
    soup = BeautifulSoup(script,'html.parser')
    search_type = staff_websites[staff_websites['School'] == school]['Website Type'].values[0]
    if search_type == 1:
        for teacher in soup.find_all('dl', {'class': 'staffPhotoWrapperRound'}):
            if name in teacher.find('dt').text.strip(): 
                return teacher.find('dd').text.strip()
    elif search_type == 2:
        for teacher in soup.find_all('div', {'class': 'portfolio-content'}):
            if name in teacher.find('h2').text.strip():
                return teacher.find('h4').text.strip()
    elif search_type == 3:
        for teacher in soup.find_all('tr', {'class': 'staff-categoryStaffMember'}):
            if name in teacher.find('a').text.strip():
                return teacher.find('span').text.strip()
    elif search_type == 4:
        for teacher in soup.find_all('div', {'class': 'tshowcase-inner-box'}):
            if name in teacher.find('div', {'class': 'tshowcase-box-title'}).text.strip():
                return teacher.find('div', {'class': 'tshowcase-single-groups'}).text.strip()
    return ''

In [7]:
soup = BeautifulSoup(open(job_page),'html.parser')
for script in soup.find_all('script'):
    if 'availJobs' in str(script):
        m = re.search('availJobs: ({.*})',str(script))
        jobs = json.loads(m.group(1))

In [8]:
#jobs['list']

In [9]:
T_F = {True: 'Y', False: 'N'}

In [10]:
jobs_df = pd.DataFrame(columns=['Nearby','Train/Bus','School','Start_date','Start_time','End_date','End_time','Multiday','Has_notes',
                                'Teacher_name','Subject','Address','Zip'])

for job in jobs['list']:
    multiday = 'N'
    if len(job['Items']) != 1:
        multiday = 'Y'
        
    school = job['Items'][0]['Institution']['Name']
    if school in all_nearby:
        nearby = 'Y'
        train_bus = 'N'
    elif school in all_train_bus:
        nearby = 'N'
        train_bus = 'Y'
    else:
        nearby = 'N'
        train_bus = 'N'
    start_date = job['Items'][0]['Start'].split('T')[0]
    start_time = job['Items'][0]['Start'].split('T')[1]
    end_date = job['Items'][-1]['End'].split('T')[0]
    end_time = job['Items'][-1]['End'].split('T')[1]
    name = job['WorkerFirstName'] + ' ' + job['WorkerLastName']
    notes = T_F[job['HasNotes']]
    address = job['Items'][0]['Institution']['Street1']
    zip_code = int(job['Items'][0]['Institution']['Zip'].strip())
    
    subject = ''
    try:
        subject = find_subject(school, name)
    except:
        pass
    
    titles = ['Regular Teacher', 'Program Option Teacher', '']
    if job['WorkerTitle'] in titles:
        jobs_df = jobs_df.append(pd.DataFrame([[nearby, train_bus, school, start_date, start_time, end_date, end_time, multiday,
                                            notes, name, subject, address, zip_code]], 
                                          columns=['Nearby', 'Train/Bus', 'School', 'Start_date', 'Start_time', 'End_date', 'End_time', 'Multiday',
                                                   'Has_notes', 'Teacher_name', 'Subject', 'Address', 'Zip']))
    
    
    
jobs_df = jobs_df.merge(zips, on='Zip').sort_values(['Nearby', 'Train/Bus'],ascending=False).reset_index(drop=True)
if date_wanted == 'all':
    tomorrow = jobs_df
else:
    tomorrow = jobs_df[jobs_df['Start_date'] == date_wanted]
    other_near = jobs_df[(jobs_df['Start_date'] != date_wanted) & ((jobs_df['Train/Bus'] == 'Y') | (jobs_df['Nearby'] == 'Y'))]
near = tomorrow[(tomorrow['Train/Bus'] == 'Y') | (tomorrow['Nearby'] == 'Y')]
far = tomorrow[(tomorrow['Train/Bus'] != 'Y') & (tomorrow['Nearby'] != 'Y')]

In [11]:
near

,Nearby,Train/Bus,School,Start_date,Start_time,End_date,End_time,Multiday,Has_notes,Teacher_name,Subject,Address,Zip,Neighborhoods
4,N,Y,Gale,2017-06-06,08:00:00,2017-06-06,15:30:00,N,N,Lanetta Wilson,,1631 West Jonquil Trail,60626.0,Rogers Park
5,N,Y,Gale,2017-06-06,08:00:00,2017-06-06,15:30:00,N,N,Danielle Washington,,1631 West Jonquil Trail,60626.0,Rogers Park


In [12]:
other_near

,Nearby,Train/Bus,School,Start_date,Start_time,End_date,End_time,Multiday,Has_notes,Teacher_name,Subject,Address,Zip,Neighborhoods
0,Y,N,Ravenswood,2017-06-05,08:45:00,2017-06-05,15:45:00,N,N,Francis Panion,,4332 North Paulina Street,60613.0,Lakeview; North Center; Uptown
1,N,Y,Courtenay,2017-06-05,07:30:00,2017-06-05,14:30:00,N,Y,Sharon Simmons,,4420 N Beacon St,60640.0,Edgewater; Lincoln Square; Uptown
2,N,Y,Courtenay,2017-06-05,07:30:00,2017-06-05,14:30:00,N,N,Adrienne Henze,,4420 N Beacon St,60640.0,Edgewater; Lincoln Square; Uptown
3,N,Y,Uplift High School,2017-06-05,08:30:00,2017-06-05,15:46:00,N,N,Fannetta Jones,,900 West Wilson Avenue,60640.0,Edgewater; Lincoln Square; Uptown
6,N,Y,Mayer,2017-06-05,08:30:00,2017-06-05,15:30:00,N,Y,Bridget Mora,,2250 North Clifton Avenue,60614.0,Lincoln Park; Logan Square
7,N,Y,Alcott Elementary/High School,2017-06-05,07:45:00,2017-06-05,15:05:00,N,Y,Jessie Mendez,,2625 North Orchard Street,60614.0,Lincoln Park; Logan Square
8,N,Y,Jenner,2017-06-05,08:00:00,2017-06-05,15:30:00,N,N,Rachel Campbell,,1119 North Cleveland Avenue,60610.0,Near North Side; Near West Side


In [13]:
far

,Nearby,Train/Bus,School,Start_date,Start_time,End_date,End_time,Multiday,Has_notes,Teacher_name,Subject,Address,Zip,Neighborhoods
17,N,N,Lyon,2017-06-06,08:00:00,2017-06-06,15:30:00,N,Y,Michelle Dunne,,2941 North McVicker Avenue,60634.0,Belmont Cragin; Dunning; Montclare; Portage Park
32,N,N,McNair,2017-06-06,07:30:00,2017-06-06,14:30:00,N,N,Moriah O'Malley,,4820 West Walton Street,60651.0,Austin; Humboldt Park
41,N,N,Monroe,2017-06-06,08:15:00,2017-06-06,15:15:00,N,N,Extra Teacher Vacancy,,3651 West Schubert Avenue,60647.0,Hermosa; Humboldt Park; Logan Square; West Town
51,N,N,Monroe,2017-06-06,08:15:00,2017-06-06,15:15:00,N,N,Extra Teacher Vacancy,,3651 West Schubert Avenue,60647.0,Hermosa; Humboldt Park; Logan Square; West Town
54,N,N,Disney II Elementary/High School,2017-06-06,08:30:00,2017-06-06,15:45:00,N,Y,Freddy Enriquez,,3815 North Kedvale Avenue,60641.0,Avondale; Belmont Cragin; Hermosa; Irving Park...
66,N,N,Beard Special Ed,2017-06-06,07:30:00,2017-06-20,14:30:00,Y,N,Extra Teacher Vacancy,,6445 West Strong Street,60656.0,O'Hare
67,N,N,Beard Special Ed,2017-06-06,07:30:00,2017-06-20,14:30:00,Y,N,Extra Teacher Vacancy,,6445 West Strong Street,60656.0,O'Hare
68,N,N,Beard Special Ed,2017-06-06,07:30:00,2017-06-20,14:30:00,Y,N,Extra Teacher Vacancy,,6445 West Strong Street,60656.0,O'Hare
72,N,N,Suder Montessori,2017-06-06,09:00:00,2017-06-06,12:30:00,N,N,Extra Teacher Vacancy,,2022 West Washington Blvd,60612.0,Near West Side; West Town
94,N,N,Reilly,2017-06-06,07:30:00,2017-06-06,14:45:00,N,N,Brenda Garcia,,3650 West School Street,60618.0,Avondale; Irving Park; North Center


In [ ]:
import geocoder
import folium

map_chicago = folium.Map(location=[41.940, -87.644], zoom_start=11)
for ind, assignment in tomorrow.iterrows():
    location = geocoder.google('%s, Chicago, IL'%assignment['Address'])
    try:
        folium.Marker(location.latlng, popup=assignment['School']).add_to(map_chicago)
    except:
        print assignment['School'], 'Not on Map'
map_chicago

In [ ]:
from google import search

for ind, assignment in tomorrow.iterrows():
    if assignment['Teacher_name'] == '':
        continue
    print '%s, %s'%(assignment['Teacher_name'], assignment['School'])
    if assignment['School'] in staff_websites.dropna(subset=['Staff Website'])['School'].values:
        print staff_websites[staff_websites['School'] == assignment['School']]['Staff Website'].values[0]
        continue
    for url in search('%s %s CPS'%(assignment['Teacher_name'],assignment['School']), tld='com', lang='en', num=3, stop=1):
        print url

Lanetta Wilson, Gale
http://schoolinfo.cps.edu/schoolprofile/schooldetails.aspx?SchoolId=609933
http://schoolreports.cps.edu/ciwp_2012/schoolid609933_esciwp_as_of_112212.pdf
http://cpspay.info/?page=49
Danielle Washington, Gale
https://www.edmodo.com/school/GaleElemCommunityAcademyIL27234
http://wgntv.com/2017/03/30/former-cps-principal-alleges-that-black-female-principals-are-being-pushed-out-of-jobs/
http://schoolinfo.cps.edu/schoolprofile/schooldetails.aspx?SchoolId=609933
Michelle Dunne, Lyon
http://cps.edu/Pages/Staff.aspx
http://schoolinfo.cps.edu/environmentalscorecard/
http://cpspay.info/?page=105
Moriah O'Malley, McNair
http://mcnair.auslchicago.org/about
http://www.omalleyandkwit.com/our-people.php
http://teacherpipeline.illinoisstate.edu/about/staff.php
 Extra Teacher Vacancy, Monroe
http://cps.edu/careers
https://cpsk12il.taleo.net/careersection/4/joblist.ftl
https://cpsk12il.taleo.net/careersection/3/jobsearch.ftl
 Extra Teacher Vacancy, Monroe
http://cps.edu/careers
https